In [2]:
!pip install PyNiteFEA==0.0.62

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install PyNiteFEA

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import PyNite
print("PyNite is installed successfully!")

PyNite is installed successfully!


In [10]:
from PyNite import FEModel3D

# ---------------------------------
# Step 1: Create the FE Model
# ---------------------------------
model = FEModel3D()

# ---------------------------------
# Step 2: Define Steel Material Properties
# ---------------------------------
E_steel = 200000  # Elastic Modulus (MPa) for Steel
nu_steel = 0.3  # Poisson's ratio for Steel
G_steel = E_steel / (2 * (1 + nu_steel))  # Shear Modulus (MPa)

# Define section properties for steel columns (I-Beam)
A_col = 0.012  # Cross-sectional area (m²) for columns
Iy_col = 0.0006  # Moment of inertia about y-axis (m⁴) for columns
Iz_col = 0.0003  # Moment of inertia about z-axis (m⁴) for columns
J_col = 0.0004  # Torsional constant (m⁴) for columns

# Define section properties for steel beams (I-Beam)
A_beam = 0.015  # Cross-sectional area (m²) for beams
Iy_beam = 0.0012  # Moment of inertia about y-axis (m⁴) for beams
Iz_beam = 0.0008  # Moment of inertia about z-axis (m⁴) for beams
J_beam = 0.0006  # Torsional constant (m⁴) for beams

# ---------------------------------
# Step 3: Define Nodes (Ground & Top)
# ---------------------------------
model.add_node("N1", 0, 0, 0)
model.add_node("N2", 4, 0, 0)
model.add_node("N3", 8, 0, 0)
model.add_node("N1_top", 0, 0, 3)
model.add_node("N2_top", 4, 0, 3)
model.add_node("N3_top", 8, 0, 3)

# ---------------------------------
# Step 4: Define Steel Column Members
# ---------------------------------
model.add_member("C1", "N1", "N1_top", E_steel, G_steel, Iy_col, Iz_col, J_col, A_col)
model.add_member("C2", "N2", "N2_top", E_steel, G_steel, Iy_col, Iz_col, J_col, A_col)
model.add_member("C3", "N3", "N3_top", E_steel, G_steel, Iy_col, Iz_col, J_col, A_col)

# ---------------------------------
# Step 5: Define Steel Beam Members
# ---------------------------------
model.add_member("B1", "N1_top", "N2_top", E_steel, G_steel, Iy_beam, Iz_beam, J_beam, A_beam)
model.add_member("B2", "N2_top", "N3_top", E_steel, G_steel, Iy_beam, Iz_beam, J_beam, A_beam)

# ---------------------------------
# Step 6: Assign Support Conditions (Fixed at Base)
# ---------------------------------
model.def_support("N1", True, True, True, True, True, True)
model.def_support("N2", True, True, True, True, True, True)
model.def_support("N3", True, True, True, True, True, True)

# ---------------------------------
# Step 7: Apply Loads (Dead Load on Beams)
# ---------------------------------
# Convert distributed load of 20 kN/m to total load over beam length
beam_length = 4  # Length of each beam in meters
total_load = -20 * beam_length  # Total load in kN
model.add_member_pt_load("B1", "Fy", total_load/2, 0.5)  # Apply point load at midspan
model.add_member_pt_load("B2", "Fy", total_load/2, 0.5)  # Apply point load at midspan

# ---------------------------------
# Step 8: Run Finite Element Analysis
# ---------------------------------
model.analyze()

# ---------------------------------
# Step 8: Run Finite Element Analysis
# ---------------------------------
try:
    model.analyze(check_statics=True)  # Added check_statics parameter
except Exception as e:
    print(f"Analysis failed: {str(e)}")
    
# ---------------------------------
# Step 9: Extract & Print Results
# ---------------------------------
print("Deflections at top nodes:")
for node in ["N1_top", "N2_top", "N3_top"]:
    try:
        # Access displacement using correct attribute
        dy = model.Nodes[node].get_displacement('dy')  # Changed to get_displacement method
        print("Node {}: DY = {:.6f} m".format(node, dy))
    except Exception as e:
        print(f"Error getting deflection for node {node}: {str(e)}")

print("\nAxial forces in columns:")
for member in ["C1", "C2", "C3"]:
    try:
        # Get axial force using correct method
        force = model.Members[member].get_axial(0)  # Changed to get_axial method
        print("Member {}: Axial Force = {:.2f} kN".format(member, force))
    except Exception as e:
        print(f"Error getting axial force for member {member}: {str(e)}")

# Add verification of results
print("\nModel Status:")
print(f"Number of nodes: {len(model.Nodes)}")
print(f"Number of members: {len(model.Members)}")

+----------------+
| Statics Check: |
+----------------+

+------------------+--------+--------+--------+--------+--------+--------+--------+---------+--------+---------+--------+---------+
| Load Combination | Sum FX | Sum RX | Sum FY | Sum RY | Sum FZ | Sum RZ | Sum MX | Sum RMX | Sum MY | Sum RMY | Sum MZ | Sum RMZ |
+------------------+--------+--------+--------+--------+--------+--------+--------+---------+--------+---------+--------+---------+
|     Combo 1      |   0    |   0    |  -80   |   80   |   0    |   0    |  240   |   -240  |   0    |    0    |  -200  |   200   |
+------------------+--------+--------+--------+--------+--------+--------+--------+---------+--------+---------+--------+---------+

Deflections at top nodes:
Error getting deflection for node N1_top: 'Node3D' object has no attribute 'get_displacement'
Error getting deflection for node N2_top: 'Node3D' object has no attribute 'get_displacement'
Error getting deflection for node N3_top: 'Node3D' object has no at

In [13]:
pip install PySimpleGUI


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [18]:
import tkinter as tk
from tkinter import messagebox
import matplotlib.pyplot as plt
from PyNite import FEModel3D

def build_fe_model():
    model = FEModel3D()
    
    # Steel material properties
    E_steel = 200000  # MPa
    nu_steel = 0.3
    G_steel = E_steel / (2 * (1 + nu_steel))
    
    # Section properties for columns (I-Beam)
    A_col = 0.012      # m²
    Iy_col = 0.0006    # m⁴
    Iz_col = 0.0003    # m⁴
    J_col = 0.0004     # m⁴
    
    # Section properties for beams (I-Beam)
    A_beam = 0.015     # m²
    Iy_beam = 0.0012   # m⁴
    Iz_beam = 0.0008   # m⁴
    J_beam = 0.0006    # m⁴
    
    # Define nodes (Ground & Top)
    model.add_node("N1", 0, 0, 0)
    model.add_node("N2", 4, 0, 0)
    model.add_node("N3", 8, 0, 0)
    model.add_node("N1_top", 0, 0, 3)
    model.add_node("N2_top", 4, 0, 3)
    model.add_node("N3_top", 8, 0, 3)
    
    # Define columns
    model.add_member("C1", "N1", "N1_top", E_steel, G_steel, Iy_col, Iz_col, J_col, A_col)
    model.add_member("C2", "N2", "N2_top", E_steel, G_steel, Iy_col, Iz_col, J_col, A_col)
    model.add_member("C3", "N3", "N3_top", E_steel, G_steel, Iy_col, Iz_col, J_col, A_col)
    
    # Define beams
    model.add_member("B1", "N1_top", "N2_top", E_steel, G_steel, Iy_beam, Iz_beam, J_beam, A_beam)
    model.add_member("B2", "N2_top", "N3_top", E_steel, G_steel, Iy_beam, Iz_beam, J_beam, A_beam)
    
    # Define fixed supports at base nodes
    for node in ["N1", "N2", "N3"]:
        model.def_support(node, True, True, True, True, True, True)
    
    # Apply loads on beams (Dead Load)
    beam_length = 4  # m
    total_load = -20 * beam_length  # kN
    model.add_member_pt_load("B1", "Fy", total_load / 2, 0.5)
    model.add_member_pt_load("B2", "Fy", total_load / 2, 0.5)
    
    # Run analysis
    try:
        model.analyze(check_statics=True)
    except Exception as e:
        print(f"Analysis failed: {e}")
    
    return model

def visualize_column(model, col_id):
    """
    Visualize the vertical deflection (dy) at the top node of the selected column.
    Assumes that the top node is named by replacing 'C' with 'N' and appending '_top'.
    """
    top_node_id = col_id.replace("C", "N") + "_top"
    try:
        dy = model.Nodes[top_node_id].get_displacement('dy')
    except Exception as e:
        messagebox.showerror("Error", f"Error retrieving displacement for {top_node_id}: {e}")
        return

    # Create a simple bar chart to display the deflection
    plt.figure(figsize=(4, 3))
    plt.bar([col_id], [dy], color='skyblue')
    plt.ylabel("Vertical Deflection (m)")
    plt.title(f"Deflection at {top_node_id}")
    lower_lim = min(dy, -0.05) - 0.05
    upper_lim = max(dy, 0.05) + 0.05
    plt.ylim(lower_lim, upper_lim)
    plt.show()

class ColumnInterface(tk.Tk):
    def __init__(self, model, columns):
        super().__init__()
        self.title("Structural Column Interface")
        self.geometry("300x250")
        self.model = model
        self.columns = columns

        # Create a label
        label = tk.Label(self, text="Reorder Columns (Use Up/Down buttons):")
        label.pack(pady=5)

        # Create a listbox to display columns
        self.listbox = tk.Listbox(self, height=5)
        self.listbox.pack(pady=5)
        self.update_listbox()

        # Create buttons for reordering and visualization
        btn_frame = tk.Frame(self)
        btn_frame.pack(pady=10)
        
        up_btn = tk.Button(btn_frame, text="Move Up", command=self.move_up)
        up_btn.grid(row=0, column=0, padx=5)
        
        down_btn = tk.Button(btn_frame, text="Move Down", command=self.move_down)
        down_btn.grid(row=0, column=1, padx=5)
        
        vis_btn = tk.Button(self, text="Visualize Selected", command=self.visualize_selected)
        vis_btn.pack(pady=5)
        
        exit_btn = tk.Button(self, text="Exit", command=self.destroy)
        exit_btn.pack(pady=5)

    def update_listbox(self):
        self.listbox.delete(0, tk.END)
        for col in self.columns:
            self.listbox.insert(tk.END, col)

    def move_up(self):
        selection = self.listbox.curselection()
        if not selection:
            messagebox.showinfo("Info", "Select a column to move.")
            return
        index = selection[0]
        if index > 0:
            self.columns[index], self.columns[index - 1] = self.columns[index - 1], self.columns[index]
            self.update_listbox()
            self.listbox.select_set(index - 1)

    def move_down(self):
        selection = self.listbox.curselection()
        if not selection:
            messagebox.showinfo("Info", "Select a column to move.")
            return
        index = selection[0]
        if index < len(self.columns) - 1:
            self.columns[index], self.columns[index + 1] = self.columns[index + 1], self.columns[index]
            self.update_listbox()
            self.listbox.select_set(index + 1)

    def visualize_selected(self):
        selection = self.listbox.curselection()
        if not selection:
            messagebox.showinfo("Info", "Please select a column to visualize.")
            return
        col_id = self.listbox.get(selection[0])
        visualize_column(self.model, col_id)

def main():
    # Build the FE model using PyNite
    model = build_fe_model()
    # Define list of column IDs
    columns = ["C1", "C2", "C3"]
    # Create and run the Tkinter interface
    app = ColumnInterface(model, columns)
    app.mainloop()

if __name__ == "__main__":
    main()


+----------------+
| Statics Check: |
+----------------+

+------------------+--------+--------+--------+--------+--------+--------+--------+---------+--------+---------+--------+---------+
| Load Combination | Sum FX | Sum RX | Sum FY | Sum RY | Sum FZ | Sum RZ | Sum MX | Sum RMX | Sum MY | Sum RMY | Sum MZ | Sum RMZ |
+------------------+--------+--------+--------+--------+--------+--------+--------+---------+--------+---------+--------+---------+
|     Combo 1      |   0    |   0    |  -80   |   80   |   0    |   0    |  240   |   -240  |   0    |    0    |  -200  |   200   |
+------------------+--------+--------+--------+--------+--------+--------+--------+---------+--------+---------+--------+---------+



In [15]:
pip show PySimpleGUI


Name: PySimpleGUI
Version: 5.0.8.1
Summary: PySimpleGUI is now located on a private PyPI server.  Please add to your pip command: -i https://PySimpleGUI.net/install
Home-page: https://www.PySimpleGUI.com
Author: PySimpleSoft Inc.
Author-email: 
License: Proprietary
Location: C:\Users\Lenovo\AppData\Roaming\Python\Python312\site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [19]:
import tkinter as tk
from tkinter import messagebox
import matplotlib.pyplot as plt
from PyNite import FEModel3D

def build_fe_model():
    model = FEModel3D()
    
    # Steel material properties
    E_steel = 200000  # MPa
    nu_steel = 0.3
    G_steel = E_steel / (2 * (1 + nu_steel))
    
    # Section properties for columns (I-Beam)
    A_col = 0.012      # m²
    Iy_col = 0.0006    # m⁴
    Iz_col = 0.0003    # m⁴
    J_col = 0.0004     # m⁴
    
    # Section properties for beams (I-Beam)
    A_beam = 0.015     # m²
    Iy_beam = 0.0012   # m⁴
    Iz_beam = 0.0008   # m⁴
    J_beam = 0.0006    # m⁴
    
    # Define nodes (Ground & Top)
    # For column C1, base: N1, top: N1_top; similarly for C2 and C3.
    model.add_node("N1", 0, 0, 0)
    model.add_node("N2", 4, 0, 0)
    model.add_node("N3", 8, 0, 0)
    model.add_node("N1_top", 0, 0, 3)
    model.add_node("N2_top", 4, 0, 3)
    model.add_node("N3_top", 8, 0, 3)
    
    # Define columns
    model.add_member("C1", "N1", "N1_top", E_steel, G_steel, Iy_col, Iz_col, J_col, A_col)
    model.add_member("C2", "N2", "N2_top", E_steel, G_steel, Iy_col, Iz_col, J_col, A_col)
    model.add_member("C3", "N3", "N3_top", E_steel, G_steel, Iy_col, Iz_col, J_col, A_col)
    
    # Define beams connecting top nodes
    model.add_member("B1", "N1_top", "N2_top", E_steel, G_steel, Iy_beam, Iz_beam, J_beam, A_beam)
    model.add_member("B2", "N2_top", "N3_top", E_steel, G_steel, Iy_beam, Iz_beam, J_beam, A_beam)
    
    # Define fixed supports at base nodes
    for node in ["N1", "N2", "N3"]:
        model.def_support(node, True, True, True, True, True, True)
    
    # Apply loads on beams (Dead Load)
    beam_length = 4  # m
    total_load = -20 * beam_length  # kN
    model.add_member_pt_load("B1", "Fy", total_load / 2, 0.5)
    model.add_member_pt_load("B2", "Fy", total_load / 2, 0.5)
    
    # Run analysis
    try:
        model.analyze(check_statics=True)
    except Exception as e:
        print(f"Analysis failed: {e}")
    
    return model

def visualize_column(model, col_id):
    """
    Visualize the vertical deflection (dy) at the top node of the selected column.
    Assumes that the top node is named by replacing 'C' with 'N' and appending '_top'.
    """
    top_node_id = col_id.replace("C", "N") + "_top"
    try:
        dy = model.Nodes[top_node_id].get_displacement('dy')
    except Exception as e:
        messagebox.showerror("Error", f"Error retrieving displacement for {top_node_id}: {e}")
        return

    plt.figure(figsize=(4, 3))
    plt.bar([col_id], [dy], color='skyblue')
    plt.ylabel("Vertical Deflection (m)")
    plt.title(f"Deflection at {top_node_id}")
    lower_lim = min(dy, -0.05) - 0.05
    upper_lim = max(dy, 0.05) + 0.05
    plt.ylim(lower_lim, upper_lim)
    plt.show()

class ColumnInterface(tk.Tk):
    def __init__(self, model, columns):
        super().__init__()
        self.title("Structural Column Repositioning")
        self.geometry("350x300")
        self.model = model
        self.columns = columns

        # Label and listbox for columns
        label = tk.Label(self, text="Select a Column to Reposition:")
        label.pack(pady=5)

        self.listbox = tk.Listbox(self, height=5)
        self.listbox.pack(pady=5)
        self.update_listbox()

        # Fields for new X and Y coordinates
        coord_frame = tk.Frame(self)
        coord_frame.pack(pady=5)
        tk.Label(coord_frame, text="New X:").grid(row=0, column=0, padx=5)
        self.entry_x = tk.Entry(coord_frame, width=10)
        self.entry_x.grid(row=0, column=1, padx=5)
        tk.Label(coord_frame, text="New Y:").grid(row=1, column=0, padx=5)
        self.entry_y = tk.Entry(coord_frame, width=10)
        self.entry_y.grid(row=1, column=1, padx=5)

        # Button to apply repositioning
        apply_btn = tk.Button(self, text="Apply Move & Recompute", command=self.apply_move)
        apply_btn.pack(pady=5)

        # Button to visualize column deflection
        vis_btn = tk.Button(self, text="Visualize Selected Column", command=self.visualize_selected)
        vis_btn.pack(pady=5)

        exit_btn = tk.Button(self, text="Exit", command=self.destroy)
        exit_btn.pack(pady=5)

    def update_listbox(self):
        self.listbox.delete(0, tk.END)
        for col in self.columns:
            self.listbox.insert(tk.END, col)

    def apply_move(self):
        selection = self.listbox.curselection()
        if not selection:
            messagebox.showinfo("Info", "Select a column first.")
            return

        col_id = self.listbox.get(selection[0])
        # Assume base node name: for column "C1", base node is "N1" and top node is "N1_top"
        base_node_id = col_id.replace("C", "N")
        top_node_id = base_node_id + "_top"

        try:
            new_x = float(self.entry_x.get())
            new_y = float(self.entry_y.get())
        except ValueError:
            messagebox.showerror("Error", "Enter valid numerical values for X and Y.")
            return

        # Update the base node coordinates
        try:
            # Save old coordinates (if needed for logging)
            old_x = self.model.Nodes[base_node_id].x
            old_y = self.model.Nodes[base_node_id].y
            self.model.Nodes[base_node_id].x = new_x
            self.model.Nodes[base_node_id].y = new_y

            # For the top node, update X and Y by the same shift while keeping Z unchanged.
            dx = new_x - old_x
            dy = new_y - old_y
            self.model.Nodes[top_node_id].x += dx
            self.model.Nodes[top_node_id].y += dy
        except Exception as e:
            messagebox.showerror("Error", f"Error updating node coordinates: {e}")
            return

        # Re-run the analysis after repositioning
        try:
            self.model.analyze(check_statics=True)
            messagebox.showinfo("Success", f"Column {col_id} repositioned.\nModel re-analyzed.")
        except Exception as e:
            messagebox.showerror("Error", f"Re-analysis failed: {e}")

    def visualize_selected(self):
        selection = self.listbox.curselection()
        if not selection:
            messagebox.showinfo("Info", "Please select a column to visualize.")
            return
        col_id = self.listbox.get(selection[0])
        visualize_column(self.model, col_id)

def main():
    # Build the FE model using PyNite
    model = build_fe_model()
    # Define list of column IDs
    columns = ["C1", "C2", "C3"]
    # Create and run the Tkinter interface
    app = ColumnInterface(model, columns)
    app.mainloop()

if __name__ == "__main__":
    main()


+----------------+
| Statics Check: |
+----------------+

+------------------+--------+--------+--------+--------+--------+--------+--------+---------+--------+---------+--------+---------+
| Load Combination | Sum FX | Sum RX | Sum FY | Sum RY | Sum FZ | Sum RZ | Sum MX | Sum RMX | Sum MY | Sum RMY | Sum MZ | Sum RMZ |
+------------------+--------+--------+--------+--------+--------+--------+--------+---------+--------+---------+--------+---------+
|     Combo 1      |   0    |   0    |  -80   |   80   |   0    |   0    |  240   |   -240  |   0    |    0    |  -200  |   200   |
+------------------+--------+--------+--------+--------+--------+--------+--------+---------+--------+---------+--------+---------+



In [25]:
import tkinter as tk
from tkinter import messagebox, ttk
import matplotlib.pyplot as plt
from PyNite import FEModel3D

# Helper to extract member end node IDs as strings
def get_member_node_ids(member):
    # Try common attribute names
    if hasattr(member, "iNode"):
        i = member.iNode
        j = member.jNode
    elif hasattr(member, "i_node"):
        i = member.i_node
        j = member.j_node
    elif hasattr(member, "i"):
        i = member.i
        j = member.j
    else:
        raise AttributeError("Member3D object has no recognized node attribute")
    # If the node is not a string, assume it's a Node3D object with an ID attribute
    if not isinstance(i, str):
        i = i.ID
    if not isinstance(j, str):
        j = j.ID
    return i, j

# --------------------------
# Build the FE Model
# --------------------------
def build_fe_model():
    model = FEModel3D()
    
    # Material properties for steel
    E_steel = 200000  # MPa
    nu_steel = 0.3
    G_steel = E_steel / (2 * (1 + nu_steel))
    
    # Section properties for columns (I-Beam)
    A_col = 0.012      # m²
    Iy_col = 0.0006    # m⁴
    Iz_col = 0.0003    # m⁴
    J_col = 0.0004     # m⁴
    
    # Section properties for beams (I-Beam)
    A_beam = 0.015     # m²
    Iy_beam = 0.0012   # m⁴
    Iz_beam = 0.0008   # m⁴
    J_beam = 0.0006    # m⁴
    
    # Define nodes with IDs as strings
    model.add_node("N1", 0, 0, 0)
    model.add_node("N2", 4, 0, 0)
    model.add_node("N3", 8, 0, 0)
    model.add_node("N1_top", 0, 0, 3)
    model.add_node("N2_top", 4, 0, 3)
    model.add_node("N3_top", 8, 0, 3)
    
    # Define columns (vertical members)
    model.add_member("C1", "N1", "N1_top", E_steel, G_steel, Iy_col, Iz_col, J_col, A_col)
    model.add_member("C2", "N2", "N2_top", E_steel, G_steel, Iy_col, Iz_col, J_col, A_col)
    model.add_member("C3", "N3", "N3_top", E_steel, G_steel, Iy_col, Iz_col, J_col, A_col)
    
    # Define beams connecting top nodes
    model.add_member("B1", "N1_top", "N2_top", E_steel, G_steel, Iy_beam, Iz_beam, J_beam, A_beam)
    model.add_member("B2", "N2_top", "N3_top", E_steel, G_steel, Iy_beam, Iz_beam, J_beam, A_beam)
    
    # Fixed supports at base nodes
    for node in ["N1", "N2", "N3"]:
        model.def_support(node, True, True, True, True, True, True)
    
    # Apply a dead load on beams
    beam_length = 4  # m
    total_load = -20 * beam_length  # kN
    model.add_member_pt_load("B1", "Fy", total_load / 2, 0.5)
    model.add_member_pt_load("B2", "Fy", total_load / 2, 0.5)
    
    # Run analysis
    try:
        model.analyze(check_statics=True)
    except Exception as e:
        print(f"Analysis failed: {e}")
    
    return model

# --------------------------
# Visualization Helpers
# --------------------------
def visualize_column_deflection(model, col_id):
    """
    Visualize the vertical deflection (dy) at the top node of the selected column.
    """
    top_node_id = col_id.replace("C", "N") + "_top"
    try:
        dy = model.Nodes[top_node_id].get_displacement('dy')
    except Exception as e:
        messagebox.showerror("Error", f"Error retrieving displacement for {top_node_id}: {e}")
        return

    plt.figure(figsize=(4, 3))
    plt.bar([col_id], [dy], color='skyblue')
    plt.ylabel("Vertical Deflection (m)")
    plt.title(f"Deflection at {top_node_id}")
    lower_lim = min(dy, -0.05) - 0.05
    upper_lim = max(dy, 0.05) + 0.05
    plt.ylim(lower_lim, upper_lim)
    plt.show()

# --------------------------
# Drawing the FE Model on Canvas
# --------------------------
class FEModelCanvas(tk.Canvas):
    def __init__(self, parent, model, width=400, height=300, **kwargs):
        super().__init__(parent, width=width, height=height, bg="white", **kwargs)
        self.model = model
        self.width = width
        self.height = height
        # Scaling factors and offsets to convert model coordinates to canvas coordinates
        self.scale = 40  # pixels per meter
        self.x_offset = 50
        self.y_offset = height - 50  # so that y increases upward in the model
        
        self.draw_model()
    
    def draw_model(self):
        self.delete("all")
        # Draw base nodes as circles and label them. Iterate over node objects.
        for node in self.model.Nodes.values():
            # Use node.ID as the identifier string
            if not node.ID.endswith("_top"):
                x = self.x_offset + node.X * self.scale
                y = self.y_offset - node.Y * self.scale
                r = 5  # circle radius
                self.create_oval(x - r, y - r, x + r, y + r, fill="blue")
                self.create_text(x, y - 10, text=node.ID, fill="black")
        
        # Draw beams connecting top nodes
        for member_id, member in self.model.Members.items():
            if member_id.startswith("B"):
                try:
                    n1_id, n2_id = get_member_node_ids(member)
                except AttributeError as e:
                    print(e)
                    continue
                # Only draw beams for nodes with IDs ending in '_top'
                if n1_id.endswith("_top") and n2_id.endswith("_top"):
                    x1 = self.x_offset + self.model.Nodes[n1_id].X * self.scale
                    y1 = self.y_offset - self.model.Nodes[n1_id].Y * self.scale
                    x2 = self.x_offset + self.model.Nodes[n2_id].X * self.scale
                    y2 = self.y_offset - self.model.Nodes[n2_id].Y * self.scale
                    self.create_line(x1, y1, x2, y2, fill="red", width=2)
        
        # Draw columns as dashed lines connecting base and top nodes.
        for member_id, member in self.model.Members.items():
            if member_id.startswith("C"):
                try:
                    base_node_id, top_node_id = get_member_node_ids(member)
                except AttributeError as e:
                    print(e)
                    continue
                x1 = self.x_offset + self.model.Nodes[base_node_id].X * self.scale
                y1 = self.y_offset - self.model.Nodes[base_node_id].Y * self.scale
                x2 = self.x_offset + self.model.Nodes[top_node_id].X * self.scale
                y2 = self.y_offset - self.model.Nodes[top_node_id].Y * self.scale
                self.create_line(x1, y1, x2, y2, fill="green", dash=(3, 3))
                self.create_text(x1, y1 + 10, text=member_id, fill="green")

# --------------------------
# Tkinter Interface with Canvas and Controls
# --------------------------
class ColumnRepositionInterface(tk.Tk):
    def __init__(self, model, columns):
        super().__init__()
        self.title("Interactive FE Model Visualization")
        self.geometry("800x400")
        self.model = model
        self.columns = columns
        
        # Create a paned window for controls and visualization
        paned = ttk.PanedWindow(self, orient=tk.HORIZONTAL)
        paned.pack(fill=tk.BOTH, expand=True)
        
        # Left frame: controls
        control_frame = ttk.Frame(paned, width=250, relief=tk.RIDGE)
        paned.add(control_frame, weight=1)
        
        # Right frame: visualization canvas
        vis_frame = ttk.Frame(paned, relief=tk.RIDGE)
        paned.add(vis_frame, weight=3)
        
        # Controls in the left frame:
        lbl = ttk.Label(control_frame, text="Select a Column to Reposition:")
        lbl.pack(pady=5)
        
        self.listbox = tk.Listbox(control_frame, height=5)
        self.listbox.pack(pady=5, fill=tk.X, padx=5)
        self.update_listbox()
        
        # Coordinate entries
        coord_frame = ttk.Frame(control_frame)
        coord_frame.pack(pady=10)
        ttk.Label(coord_frame, text="New X:").grid(row=0, column=0, padx=5, pady=2)
        sel


In [27]:
import tkinter as tk
from tkinter import messagebox, ttk
import matplotlib.pyplot as plt
from PyNite import FEModel3D

# Helper to extract member end node IDs as strings
def get_member_node_ids(member):
    # Try common attribute names
    if hasattr(member, "iNode"):
        i = member.iNode
        j = member.jNode
    elif hasattr(member, "i_node"):
        i = member.i_node
        j = member.j_node
    elif hasattr(member, "i"):
        i = member.i
        j = member.j
    else:
        raise AttributeError("Member3D object has no recognized node attribute")
    # Convert to strings if necessary
    return str(i), str(j)

# --------------------------
# Build the FE Model
# --------------------------
def build_fe_model():
    model = FEModel3D()
    
    # Material properties for steel
    E = 200000  # MPa
    nu = 0.3
    G = E / (2 * (1 + nu))
    
    # Section properties (example values)
    A = 0.012
    Iy = 0.0006
    Iz = 0.0003
    J = 0.0004
    
    # Add nodes (base and top) with string IDs
    model.add_node("N1", 0, 0, 0)
    model.add_node("N2", 4, 0, 0)
    model.add_node("N3", 8, 0, 0)
    model.add_node("N1_top", 0, 0, 3)
    model.add_node("N2_top", 4, 0, 3)
    model.add_node("N3_top", 8, 0, 3)
    
    # Add members (columns and beams)
    model.add_member("C1", "N1", "N1_top", E, G, Iy, Iz, J, A)
    model.add_member("C2", "N2", "N2_top", E, G, Iy, Iz, J, A)
    model.add_member("C3", "N3", "N3_top", E, G, Iy, Iz, J, A)
    model.add_member("B1", "N1_top", "N2_top", E, G, Iy, Iz, J, A)
    model.add_member("B2", "N2_top", "N3_top", E, G, Iy, Iz, J, A)
    
    # Define supports at base nodes
    for node in ["N1", "N2", "N3"]:
        model.def_support(node, True, True, True, True, True, True)
    
    # Apply a simple load on beams (example)
    model.add_member_pt_load("B1", "Fy", -40, 0.5)
    model.add_member_pt_load("B2", "Fy", -40, 0.5)
    
    try:
        model.analyze(check_statics=True)
    except Exception as e:
        print(f"Analysis failed: {e}")
    
    return model

# --------------------------
# Minimal visualization: draw FE model on a Tkinter Canvas
# --------------------------
class FEModelCanvas(tk.Canvas):
    def __init__(self, parent, model, width=500, height=350, **kwargs):
        super().__init__(parent, width=width, height=height, bg="white", **kwargs)
        self.model = model
        self.width = width
        self.height = height
        self.scale = 40       # pixels per meter
        self.x_offset = 50
        self.y_offset = height - 50  # so that y increases upward
        self.draw_model()
    
    def draw_model(self):
        self.delete("all")
        print("Drawing FE model on canvas...")
        # Draw base nodes as circles and label them
        for node in self.model.Nodes.values():
            node_id = str(node.ID)
            if not node_id.endswith("_top"):
                x = self.x_offset + node.X * self.scale
                y = self.y_offset - node.Y * self.scale
                print(f"Drawing node {node_id} at ({x}, {y})")
                r = 5
                self.create_oval(x - r, y - r, x + r, y + r, fill="blue")
                self.create_text(x, y - 10, text=node_id, fill="black")
        # Draw beams (assume member IDs starting with "B")
        for member in self.model.Members.values():
            member_id = str(member.ID)
            if member_id.startswith("B"):
                try:
                    n1_id, n2_id = get_member_node_ids(member)
                except AttributeError as e:
                    print(e)
                    continue
                # Draw only if both end IDs end with "_top"
                if str(n1_id).endswith("_top") and str(n2_id).endswith("_top"):
                    x1 = self.x_offset + self.model.Nodes[n1_id].X * self.scale
                    y1 = self.y_offset - self.model.Nodes[n1_id].Y * self.scale
                    x2 = self.x_offset + self.model.Nodes[n2_id].X * self.scale
                    y2 = self.y_offset - self.model.Nodes[n2_id].Y * self.scale
                    self.create_line(x1, y1, x2, y2, fill="red", width=2)
        # Draw columns (assume member IDs starting with "C")
        for member in self.model.Members.values():
            member_id = str(member.ID)
            if member_id.startswith("C"):
                try:
                    base_node_id, top_node_id = get_member_node_ids(member)
                except AttributeError as e:
                    print(e)
                    continue
                x1 = self.x_offset + self.model.Nodes[base_node_id].X * self.scale
                y1 = self.y_offset - self.model.Nodes[base_node_id].Y * self.scale
                x2 = self.x_offset + self.model.Nodes[top_node_id].X * self.scale
                y2 = self.y_offset - self.model.Nodes[top_node_id].Y * self.scale
                self.create_line(x1, y1, x2, y2, fill="green", dash=(3,3))
                self.create_text(x1, y1 + 10, text=member_id, fill="green")

# --------------------------
# Main interface that shows the canvas
# --------------------------
class FEInterface(tk.Tk):
    def __init__(self, model):
        super().__init__()
        self.title("FE Model Visualization")
        self.geometry("800x400")
        self.model = model
        
        # Create a frame for the canvas
        canvas_frame = tk.Frame(self)
        canvas_frame.pack(fill=tk.BOTH, expand=True)
        
        self.canvas = FEModelCanvas(canvas_frame, self.model, width=500, height=350)
        self.canvas.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)
        
        # Button to force a redraw
        redraw_btn = tk.Button(self, text="Redraw Canvas", command=self.canvas.draw_model)
        redraw_btn.pack(pady=5)
        
        print("GUI is launching...")
    
def main():
    model = build_fe_model()
    app = FEInterface(model)
    print("Entering main loop.")
    app.mainloop()

if __name__ == "__main__":
    main()


+----------------+
| Statics Check: |
+----------------+

+------------------+--------+--------+--------+--------+--------+--------+--------+---------+--------+---------+--------+---------+
| Load Combination | Sum FX | Sum RX | Sum FY | Sum RY | Sum FZ | Sum RZ | Sum MX | Sum RMX | Sum MY | Sum RMY | Sum MZ | Sum RMZ |
+------------------+--------+--------+--------+--------+--------+--------+--------+---------+--------+---------+--------+---------+
|     Combo 1      |   0    |   0    |  -80   |   80   |   0    |   0    |  240   |   -240  |   0    |    0    |  -200  |   200   |
+------------------+--------+--------+--------+--------+--------+--------+--------+---------+--------+---------+--------+---------+

Drawing FE model on canvas...
Drawing node 0 at (50, 300)
Drawing node 1 at (210, 300)
Drawing node 2 at (370, 300)
Drawing node 3 at (50, 300)
Drawing node 4 at (210, 300)
Drawing node 5 at (370, 300)
GUI is launching...
Entering main loop.
+----------------+
| Statics Check: |
+

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Lenovo\AppData\Local\Temp\ipykernel_30892\1990191430.py", line 252, in apply_move
    self.canvas.draw_model()
    ^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\tkinter\__init__.py", line 2433, in __getattr__
    return getattr(self.tk, attr)
           ^^^^^^^^^^^^^^^^^^^^^^
AttributeError: '_tkinter.tkapp' object has no attribute 'canvas'
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Lenovo\AppData\Local\Temp\ipykernel_30892\1990191430.py", line 252, in apply_move
    self.canvas.draw_model()
    ^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\tkinter\__init__.py", line 2433, in __getattr__
  

+----------------+
| Statics Check: |
+----------------+

+------------------+--------+--------+--------+--------+--------+----------+--------+---------+--------+---------+--------+---------+
| Load Combination | Sum FX | Sum RX | Sum FY | Sum RY | Sum FZ |  Sum RZ  | Sum MX | Sum RMX | Sum MY | Sum RMY | Sum MZ | Sum RMZ |
+------------------+--------+--------+--------+--------+--------+----------+--------+---------+--------+---------+--------+---------+
|     Combo 1      |  41.9  | -41.9  |  -40   |   40   |   0    | 5.55e-16 |  120   |   -120  |  126   |   -126  |  -80   |    80   |
+------------------+--------+--------+--------+--------+--------+----------+--------+---------+--------+---------+--------+---------+



Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Lenovo\AppData\Local\Temp\ipykernel_30892\1990191430.py", line 252, in apply_move
    self.canvas.draw_model()
    ^^^^^^^^^^^
  File "c:\ProgramData\anaconda3\Lib\tkinter\__init__.py", line 2433, in __getattr__
    return getattr(self.tk, attr)
           ^^^^^^^^^^^^^^^^^^^^^^
AttributeError: '_tkinter.tkapp' object has no attribute 'canvas'


Drawing FE model on canvas...
Drawing node 0 at (50, 300)
Drawing node 1 at (210, 300)
Drawing node 2 at (370, 300)
Drawing node 3 at (50, 300)
Drawing node 4 at (210, 300)
Drawing node 5 at (370, 300)


KeyboardInterrupt: 